In [ ]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [ ]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [9]:
%matplotlib inline

import pandas as pd
import numpy as np

from sklearn.naive_bayes import GaussianNB

In [14]:
csv = pd.read_csv("matches.csv", header=0, encoding="latin-1")
print(csv.shape)

(34256, 14)


In [15]:
x = np.array(csv[["team1Text", "team2Text"]])
y = np.array(csv["resText"])

print(x.shape)
print(y.shape)

(34256, 2)
(34256,)


In [12]:
model = GaussianNB()
model.fit(x, y)

ValueError: could not convert string to float: 'France'